# hiseq.align


The standard port for aligner

## 1 Requirements  


### 1.1 Arguments

  - input: str 
  - index: str
  - outdir: pwd/str
  - unique: True/False
  - aligner: (guess aligner) 
  - smp_name: None/str
  - index_name: None/str
  - threads: 4/int
  - overwrite: True/False
  - extra_para: (in dict, toml)


### 1.2 Output

  - config.toml
  - bam  
  - log  
  - stat 
  - cmd.sh 
  - unmap  
  - flagstat 


### 1.3 Directory structure 

  - outdir/smp_name/index_name/files



## 2. Interpreter 



### 2.1 Functions 

  - unique|multiple, determine the aligner 
  

### 2.2 Output

  - required arguments for `Align` 




## 3 Aligner


### 3.1 Bowtie   

```
# common: -p 4 --mm
# default: k=1
bowtie -k 1 -S -v 2 --best --un unmap.fq -x index se.fq 1> align.sam 2> align.log
bowtie -k 1 -S -v 2 --best --un unmap.fq -x index -1 pe_1.fq -2 pe_2.fq 1> align.sam 2> align.log

# unique
bowtie -m 1 -S -v 2 --best --un unmap.fq -x index se.fq 1> align.sam 2> align.log
bowtie -m 1 -S -v 2 --best --un unmap.fq -x index -1 pe_1.fq -2 pe_2.fq 1> align.sam 2> align.log
```

### 3.2 Bowtie2 

```
# common: -p 4 --mm
# default:
bowtie2 --very-sensitive --no-unal -x index -U se.fq --un unmap.fq 1> align.sam 2> align.log
bowtie2 --very-sensitive --no-unal --no-mixed --no-discordant --un-conc unmap.fq 1> align.sam 2> align.log

# unique
# https://www.biostars.org/p/101533/#101537, -q 5,10
samtools view -bhs -q 10 align.bam 


# extra parameters
-I minimum fragment length (0) 
-X maximum fragment length (500) 
```


### 3.3 STAR 

```
# common
# default:
STAR \
    --genomeLoad NoSharedMemory \
    --runMode alignReads \
    --genomeDir index \
    --readFilesIn pe_1 pe_2 \
    --readFilesCommand zcat \
    --outFileNamePrefix smp_name \
    --runThreadN 4 \
    --limitBAMsortRAM 10000000000 \
    --outSAMtype BAM SortedByCoordinate \
    --outFilterMismatchNoverLmx 0.07 \
    --seedSearchStartLmax 20 \
    --outReadsUnmapped Fastx 

# unique:
--outFilterMultimapNmax 1

# small genome (default: 50)
--seedPerWindowNmax 5


# For sharing memory in STAR
# by Devon Ryan: https://www.biostars.org/p/260069/#260077 
# by Dobin: https://github.com/alexdobin/STAR/pull/26
# --genomeLoad
```


















Updates:

1. style the code with PEP8: https://www.python.org/dev/peps/pep-0008/

in brief:

1 indentation: 4 spaces  
    1.1 Aligned with opening delimiter
2 Maximum Line Length: 79 characters 
    2.1 Multi lines, operator at end
3 Blank Lines
    3.1 Two blank lines surround top-level function/class  
    3.2 Single blank line, in class  
    3.3 
4 White spaces  
    4.1 no spaces after/before brackets/braces  
    4.2 single space on both sides of binary operator   
    4.3 no spaces around '=' for keyword argument  

5 Naming styles

5.1 lower_case_with_underscores, 

6 Package and Module Names 

6.1 Modules should have short, all-lowercase names

7 Class names should normally use the CapWords convention. 

8 Function and Variable Names,  lowercase, with words separated by underscores as necessary to improve readability.

9 Use `is`, `is not`

10 Use ''.startswith(), ''.endswith()



















In [ ]:
%reload_ext autoreload
%autoreload 2

import aligner_index
from aligner_index import *
from utils import *

args = {
    'genome': 'mm10',
    'aligner': 'bowtie',
    # 'spikein': 'dm3',
    # 'to_rRNA': True,
    'to_chrM': True,
    # 'to_MT_trRNA': True,
    'verbose': True,
}

# check_index_args(**args)

# fetch_index('dm6', 'chrM', 'bowtie')

# x = '/home/wangming/data/genome/dm6/bowtie_index/chrM'
# aligner = 'bowtie'

# p = AlignIndex(x, aligner)

# p.is_bowtie_index()

In [ ]:
%reload_ext autoreload
%autoreload 2

from bowtie2 import *
from star import *


args = {
    'fq1': '/data/yulab/wangming/work/devel_pipeline/hiseq/rnaseq/data/pe_control_rep1_1.fq.gz',
    # 'fq2': '/data/yulab/wangming/work/devel_pipeline/hiseq/rnaseq/data/pe_control_rep1_2.fq.gz',
    'index': '/home/wangming/data/genome/dm6/STAR_index/dm6',
    'outdir': 'test',
    'genome': 'dm6',
    'threads': 24,
    'unique_only': True,
}

# a = BowtieConfig(**args)
# check_fx_args(args['fq1'], args['fq2'])


# a = Bowtie2(**args)
# a.run()


a = Star(**args)
a.run()


# x = 'test/pe_control_rep1/chrM/pe_control_rep1.align.log'
# parse_bowtie2(x)


# x = '/home/wangming/data/genome/dm6/STAR_index/chrM'
# a = AlignIndex(x, 'STAR')
# a.is_valid()


In [24]:
%reload_ext autoreload
%autoreload 2
import os
import fnmatch
import logging as log

## 1. files and path
def listdir(x, full_name=True, recursive=False, include_dir=False):
    """List all the files within the path
    see: list.dir() in R
    see answers on :https://stackoverflow.com/a/3207973
    
    Parameters
    ----------
    x : str
        List files (dirs) in x
        
    full_name : bool
        Return the fullname of the files/dirs 
        
    recursive : bool
        List files/dirs recursively 
        
    include_dir : bool
        Return the dirs
    """
    out = []
    if isinstance(x, str):
        if os.path.isdir(x):
            n = 0 # levels
            for (root, d, f) in os.walk(x):
                dirs = [os.path.join(root, i) for i in d] if full_name else d
                files = [os.path.join(root, i) for i in f] if full_name else f
                out += files
                if include_dir:
                    out += dirs
                if not recursive:
                    break # first level
        else:
            log.error('listdir() skipped, x not a directory: {}'.format(x))
    else:
        log.error('listdir() skipped, x expect str, got {}'.format(
            type(x).__name__))
    return out


def listfile(path='.', pattern='*', full_name=True, recursive=False,
    include_dir=False):
    """Search files by the pattern, within directory
    fnmatch.fnmatch()

    pattern:
    *       matches everything
    ?       matches any single character
    [seq]   matches any character in seq
    [!seq]  matches any char not in seq

    An initial period in FILENAME is not special.
    Both FILENAME and PATTERN are first case-normalized
    if the operating system requires it.
    If you don't want this, use fnmatchcase(FILENAME, PATTERN).

    example:
    listfile('./', '*.fq')
    """
    files = listdir(path, full_name, recursive, include_dir)
    files = [f for f in files if fnmatch.fnmatch(os.path.basename(f), pattern)]
    return sorted(files)


x = 'test/data/'
listfile(x, '*_1.fq.gz', recursive=True)


['test/data/pe_control_rep1_1.fq.gz',
 'test/data/pe_control_rep2_1.fq.gz',
 'test/data/pe_treatment_rep1_1.fq.gz',
 'test/data/pe_treatment_rep2_1.fq.gz']

In [94]:
import os
import re
import shutil
import logging as log
from utils import *


def gzip_file(src, dest=None, decompress=True, **kwargs):
    """Gzip Compress or Decompress files using gzip module in python
    rm, True/False, whether remove old file

    # check the src file by extension: .gz
    """
    show_log = kwargs.get('show_log', True)
    show_error = kwargs.get('show_error', False)
    compresslevel = kwargs.get('compresslevel', 1)
    threads = kwargs.get('threads', 4)
    flag = False
    # input: src
    if not isinstance(src, str):
        if show_error:
            log.error('src expect str, got {}'.format(src))
    if not file_exists(src):
        if show_error:
            log.error('src not exists, {}'.format(src))
    # output: dest
    if dest is None:
        dest = os.path.splitext(src)[0] if decompress else src + '.gz'
    if isinstance(dest, str):
        if file_exists(dest):
            if show_error:
                log.error('dest exists, {}'.format(dest))
        elif os.path.exists(os.path.dirname(dest)):
            flag = True
        else:
            if show_error:
                log.error('dest not valid, {}'.format(dest))
    else:
        if show_error:
            log.error('dest expect str, got {}'.format(dest))
    # do-the-thing
    out = None
    if flag:
        is_gzipped = file_is_gzipped(src)
        if decompress:
            if is_gzipped:
                with xopen(src, 'rb') as r, \
                    xopen(dest, 'wb') as w:
                    shutil.copyfileobj(r, w)
                out = dest
            else:
                if show_error:
                    log.error('src is not gzipped, skipped. {}'.format(src))
        else:
            if is_gzipped:
                if show_error:
                    log.error('src is gzipped, skipped. {}'.format(src))
            else:
                with xopen(src, 'rb') as r, \
                    xopen(dest, 'wb', threads=threads,
                          compresslevel=compresslevel) as w:
                    shutil.copyfileobj(r, w)
                out = dest
    return out



x = 'test/data/pe_control_rep1_1.fq.gz'
y = 'test/aaa.fq'

gzip_file(x, y)


NameError: name 'file_is_gzipped' is not defined

In [91]:
from xopen import xopen

help(xopen)

Help on function xopen in module xopen:

xopen(filename, mode: str = 'r', compresslevel: Union[int, NoneType] = None, threads: Union[int, NoneType] = None) -> <class 'IO'>
    A replacement for the "open" function that can also read and write
    compressed files transparently. The supported compression formats are gzip,
    bzip2 and xz. If the filename is '-', standard output (mode 'w') or
    standard input (mode 'r') is returned.
    
    The file type is determined based on the filename: .gz is gzip, .bz2 is bzip2, .xz is
    xz/lzma and no compression assumed otherwise.
    
    mode can be: 'rt', 'rb', 'at', 'ab', 'wt', or 'wb'. Also, the 't' can be omitted,
    so instead of 'rt', 'wt' and 'at', the abbreviations 'r', 'w' and 'a' can be used.
    
    In Python 2, the 't' and 'b' characters are ignored.
    
    Append mode ('a', 'at', 'ab') is not available with BZ2 compression and
    will raise an error.
    
    compresslevel is the compression level for writing to gzip fil